In [1]:
var ds = require('dstools');
var input = require('../examples/data/3features/input-m.json'); 
var diversibute = require('../index');

#### Wie schneiden die beiden Algorithmen bei 100x Tests ab?  

Gruppengröße: 26  
Gruppenanzahl: 3

In [2]:
var groups = 3;

In [ ]:
// 100x runs of monte carlo
var scoresMonteCarlo = [];
var resultsMonteCarlo = {};

// Initialize empty promise collection
var runs = [];

for(let run = 0; run < 100; run++) {
    
    // Calculate groups with monte carlo
    runs.push(diversibute.monteCarlo(input, groups).then(result => {
                
        // Collect result and score
        resultsMonteCarlo[result.score] = result;
        scoresMonteCarlo.push(result.score);
        
        return;
    }));
}

var highscoreMonteCarlo = undefined;

// Run async tasks in a sequence
runs.reduce((promiseChain, currentTask) => {    
    return promiseChain.then(chainResults =>        
             currentTask.then(currentResult =>            
                  [ ...chainResults, currentResult ]        
             )    
        );
    },Promise.resolve([])).then(arrayOfResults => {    

        // Do something with all results
        console.log(scoresMonteCarlo);

        // Calculate highest score of this batch
        highscoreMonteCarlo = Math.max(...scoresMonteCarlo);

        console.log(highscoreMonteCarlo);
    });

In [5]:
// 100x runs of genetic
var scoresGenetic = [];
var resultsGenetic = {};
var runs = [];

// Run algorithm x times
for(let run = 0; run < 100; run++) {
    runs.push(diversibute.genetic(input, groups).then(result => {
        
        // Collect result and score
        resultsGenetic[result.score] = result;
        return scoresGenetic.push(result.score);
    }));
}

var highscoreGenetic = undefined;

// We must wait until all calculations are done
Promise.all(runs).then(values => {
    
    console.log(scoresGenetic);

    // Calculate highest score of this batch
    highscoreGenetic = Math.max(...scoresGenetic);

    console.log(highscoreGenetic);
});

217.2464129550104
217.2464129550104
217.2464129550104
217.2464129550104
217.2464129550104
217.2464129550104
217.2464129550104
217.2464129550104
217.2464129550104
217.2464129550104
[ 217.2464129550104 ]
217.2464129550104


## Vergleich Algorithmen

In [ ]:
// Calculate overall hitrate

// Highest score
var highestScore = (highscoreMonteCarlo > highscoreGenetic) ? highscoreMonteCarlo : highscoreGenetic;

var hitRateMonteCarlo = scoresMonteCarlo.map(v => v/highestScore);
var hitRateGenetic = scoresGenetic.map(v => v/highestScore);

In [ ]:
var trace1 = {
  x: hitRateMonteCarlo,
  type: "histogram",
  opacity: 0.5,
  marker: {
     color: 'green',
  },
  name: 'Monte Carlo'
};
var trace2 = {
  x: hitRateGenetic,
  type: "histogram",
  opacity: 0.6,
  marker: {
     color: 'red',
  },
  name: 'Genetic Algorithm'
};

var data = [trace1, trace2];
var layout = {barmode: "overlay", xaxis: {title: 'Trefferquote'}, yaxis: {title: 'Häufigkeit'}};

ds.Collection().plotly(data, layout).show();

## Darstellung "bestes" Ergebnis

In [ ]:
// Retrieve winner combination
var winnerSet = (highestScore == highscoreMonteCarlo) ? resultsMonteCarlo : resultsGenetic;
var winner = winnerSet[highestScore];

// Generate empty traces
/*var traces = ['rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)'].map(c => {
    return {x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
            color: c,
            width: 0.5},
            opacity: 0.8},
        type: 'scatter3d'};
});
*/
var traces = [];
for(let i = 0; i < groups; i++) {
    traces.push({x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5
            },
            opacity: 0.8
        },
        type: 'scatter3d'
    });
}

// Loop input data and build group traces
Object.keys(input).forEach((member, index) => {
    
    // Retrieve member group 
    var group = winner.combination[index];
    
    traces[group-1].x.push(input[member][0]);
    traces[group-1].y.push(input[member][1]);
    traces[group-1].z.push(input[member][2]);    
});

// Print 3D Model with group traces and colors
ds.Collection().plotly(traces,{margin: {l: 0, r: 0, b: 0, t: 0 }}).show();